# Process Data from 2016 into a consistent format.

> This notebook brings the 2016 into alignment with the desired format with respect to field name, type, and grouping.

In [ ]:
# Imports ----
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
import pickle

In [ ]:
from g2fd.internal import *

In [ ]:
#| default_exp internal

In [ ]:
# 2016
year_string = '2016'

meta_path = './data/raw/G2F_Planting_Season_2016_v2/z._2016_supplemental_info/g2f_2016_field_metadata.csv' 
phno_path = './data/raw/G2F_Planting_Season_2016_v2/a._2016_hybrid_phenotypic_data/g2f_2016_hybrid_data_clean.csv' 
# geno_path = None,  
wthr_path = './data/raw/G2F_Planting_Season_2016_v2/b._2016_weather_data/g2f_2016_weather_data.csv'
soil_path = './data/raw/G2F_Planting_Season_2016_v2/c._2016_soil_data/g2f_2016_soil_data_clean.csv'
mgmt_path = './data/raw/G2F_Planting_Season_2016_v2/z._2016_supplemental_info/g2f_2016_agronomic_information.csv'

meta = pd.read_csv(meta_path, encoding = "ISO-8859-1", low_memory=False)
phno = pd.read_csv(phno_path, encoding = "ISO-8859-1", low_memory=False)
wthr = pd.read_csv(wthr_path, encoding = "ISO-8859-1", low_memory=False)
soil = pd.read_csv(soil_path, encoding = "ISO-8859-1", low_memory=False)
mgmt = pd.read_csv(mgmt_path, encoding = "ISO-8859-1", low_memory=False)

In [ ]:
# load dicts for column renaming
meta_name_dict = mk_name_dict(name = 'meta')
phno_name_dict = mk_name_dict(name = 'phno')
soil_name_dict = mk_name_dict(name = 'soil')
wthr_name_dict = mk_name_dict(name = 'wthr')
mgmt_name_dict = mk_name_dict(name = 'mgmt')

# Rename
**Naming rules:**
- One dict for each input df
- Comment out anything that shouldn't be changed
- Upper_Upper_Unit_\$unit
- Upper_$number
- No special characters


In [ ]:
(find_unrecognized_columns(df = meta, dct = meta_name_dict),
find_unrecognized_columns(df = phno, dct = phno_name_dict),
find_unrecognized_columns(df = soil, dct = soil_name_dict),
find_unrecognized_columns(df = wthr, dct = wthr_name_dict),
find_unrecognized_columns(df = mgmt, dct = mgmt_name_dict))

([], [], [], [], [])

In [ ]:
meta = meta.rename(columns=meta_name_dict)
phno = phno.rename(columns=phno_name_dict)
soil = soil.rename(columns=soil_name_dict)
wthr = wthr.rename(columns=wthr_name_dict)
mgmt = mgmt.rename(columns=mgmt_name_dict)

# add indicator columns to help with debugging merge
meta['meta'] = True
phno['phno'] = True
soil['soil'] = True
wthr['wthr'] = True
mgmt['mgmt'] = True

In [ ]:
[e.shape for e in [meta, phno, soil, wthr, mgmt]]

[(29, 52), (16377, 39), (37, 34), (250480, 27), (141, 9)]

# Sanatize ID columns as needed


In [ ]:
# confirm everything's okay
print(
  'meta', find_unrecognized_experiments(meta.Experiment_Code, return_all_exps=False), 
'\nphno', find_unrecognized_experiments(phno.Experiment_Code, return_all_exps=False),
'\nsoil', find_unrecognized_experiments(soil.Experiment_Code, return_all_exps=False),
'\nwthr', find_unrecognized_experiments(wthr.Experiment_Code, return_all_exps=False),
'\nmgmt', find_unrecognized_experiments(mgmt.Experiment_Code, return_all_exps=False),
'\nall ', find_unrecognized_experiments([], return_all_exps=True)
)  

meta [] 
phno [] 
soil [] 
wthr [] 
mgmt [] 
all  ['ARH1', 'ARH2', 'AZH1', 'COH1', 'DEH1', 'GAH1', 'GAH2', 'GEH1', 'GEH2', 'IAH1', 'IAH2', 'IAH2 ', 'IAH3', 'IAH3 ', 'IAH4', 'IAH4 ', 'ILH1', 'ILH2', 'INH1', 'KSH1', 'KSH2', 'KSH3', 'MIH1', 'MNH1', 'MOH1', 'MOH1 ', 'MOH1-Rep1', 'MOH1-Rep2', 'NCH1', 'NEH1', 'NEH2', 'NEH3', 'NEH4', 'NYH1', 'NYH1', 'NYH2', 'NYH3', 'NYH4', 'NYS1', 'OHH1', 'ONH1', 'ONH2', 'SCH1', 'TXH1', 'TXH1-Dry', 'TXH1-Early', 'TXH1-Late', 'TXH2', 'TXH3', 'TXH4', 'W1H1', 'W1H2', 'WIH1', 'WIH2', 'WIH3']


# Rearrange columns

In [ ]:
# separate static and dynamic values
sval = phno.merge(soil, how = 'outer')
sval = sval.merge(meta, how = 'outer') # This introduces 3 sites that have no data
# sval.shape # used to confirm nrow = #20574 + 3

# these tables are different enought we'll keep them separate
# mgmt
# unfortunately we need multiples because at least one field treats different passes differently
mgmt = phno.loc[:, ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'Replicate', 'phno']
#                 ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'phno']
               ].drop_duplicates().merge(mgmt, how = 'outer')
# confirm there are no rows in mgmt that are not in phno
temp = mgmt.loc[(~mgmt.phno & mgmt.mgmt), :]
if 0 != temp.shape[0]:
    print(temp)
else:
    mgmt = mgmt.loc[mgmt.mgmt.notna(), :].drop(columns = 'phno')


# wthr
# There's only ever one weather station so we have to worry about imputation but not duplicates

In [ ]:
# Set each id col to a string
        # 'Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'Replicate'
for i in ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot']:
    sval[i] = sval[i].astype('string')
    mgmt[i]  =  mgmt[i].astype('string')
    
    if i not in ['Range', 'Pass', 'Plot']:
        wthr[i]  =  wthr[i].astype('string')

# Sanitize Non-ID columns



## Sanitization functions

The pattern to use is:
 1. Alter the dataframe
 1. Test the dataframe against expectations
 
The main tasks that need to be completed are:
 1. Identify values that can't be converted to the expected data type. The "find_unconvertable_" family of functions should be used. 
     1. `find_unconvertable_datetimes`
     
 1. For simple renaming (e.g. misspellings) or splitting non-tidy data into two rows ("entry1-entry2" -> "entry1", "entry2") use `sanitize_col` 
 1. Move values that are ambigous but pertain to data imputation to "Imputation_Notes" using `relocate_to_Imputation_Notes`
 1. If new columns need to be added (e.g. mgmt.Ingredient for parsed components of Product (e.g. elements) ) this should be accomplished with `safe_create_col`.
 1. Any one off changes should be accomplised manually. 
 1. Confirm columns match the expected types with `check_df_dtype_expectations`, and report mismatches. 


These steps should be completed for each dataframe in turn to minimize the cognitive load of the reader. 

## Sanitization: Column data type expectations
Note: to handle missing values some columns that would otherwise be ints are floats

In [ ]:
sval_col_dtypes = mk_dtype_dict(name = 'sval')
wthr_col_dtypes = mk_dtype_dict(name = 'wthr')
mgmt_col_dtypes = mk_dtype_dict(name = 'mgmt')

# Sanitization: Alter entries

## Static values (within season)

### Datetime containing columns

In [ ]:
sval.loc[sval.Recieved_Date_Unit_Datetime == 'Local Lab', 'Recieved_Date_Unit_Datetime'] = np.nan

In [ ]:
# convert the date cols into datetime. Lean on pd.to_datetime() to infer the format, assume that each site uses the same format.

for e in ['Planted_Unit_Datetime', 
    'Harvested_Unit_Datetime', 
    'Anthesis_Unit_Datetime', 
    'Silking_Unit_Datetime', 
    'Recieved_Date_Unit_Datetime', 
    'Processed_Date_Unit_Datetime', 
    'Weather_Station_Placed_Unit_Datetime', 
    'Weather_Station_Removed_Unit_Datetime'
    ]:
#     print(e)
#     print(find_unconvertable_datetimes(df_col=sval[e], pattern='%Y-%m-%d %H:%M', index=False))

    sval['Datetime_Temp'] = pd.to_datetime(np.nan)

    for code in list(sval.Experiment_Code.drop_duplicates()):
        sval.loc[sval.Experiment_Code == code, 'Datetime_Temp'
                 ] = pd.to_datetime(sval.loc[sval.Experiment_Code == code, e])

    sval.loc[:, e] = sval.loc[:, 'Datetime_Temp'] 

sval = sval.drop(columns = 'Datetime_Temp')

In [ ]:
# # -> floats
sval = sanitize_col(
    df = sval, 
    col = 'Soil_1_to_1_Unit_pH', 
    simple_renames= {'1:2 soil:water 7': '7', 
                     '1:2 soil:water 6.5': '6.5'
                    }, 
    split_renames= {})


# Cation_Exchange_Capacity is listed as "CEC/Sum of Cations me/100g" 
# If these are adjusted then the numbers will be notably lower than the other values
# Droping these values and will impute them.
for e in ['296 umho/cm', '269 umho/cm', '14.05 cmolc/kg', '9 cmocc/kg']:
    sval.loc[sval.Cation_Exchange_Capacity == e, 'Cation_Exchange_Capacity'] = np.nan


sval.loc[sval.Pounds_Needed_Soil_Moisture == 'Unknown', 'Pounds_Needed_Soil_Moisture'] = np.nan
sval = sanitize_col(
    df = sval, 
    col = 'Pounds_Needed_Soil_Moisture', 
    simple_renames= {'<1': '1', 
                     '~5 lbs': '5', 
                     '2.5 lbs.': '2.5'
                    }, 
    split_renames= {})

In [ ]:
# convert types
for e in ['Alley_Length_Unit_Inches', 'Row_Spacing_Unit_Inches', 
         'Anthesis_Unit_Days', 'Silking_Unit_Days', 'Kernels_Per_Plot',
         'Soil_1_to_1_Unit_pH', 'Cation_Exchange_Capacity', 'Pounds_Needed_Soil_Moisture']:
    err_list = find_unconvertable_numerics(df_col = sval[e], index = False)
    if err_list != []:
        print(e)
        print(err_list)
    else:
        sval[e] = sval[e].astype('float')

In [ ]:
# to bool
sval = sanitize_col(
    df = sval, 
    col = 'Discarded', 
    simple_renames= {
        'Yes':'True',
        'yes':'True'}, 
    split_renames= {})

# set missing to false
sval.loc[sval.Discarded.isna(), 'Discarded'] = 'False'
sval.Discarded = sval.Discarded.map({'True': True, 'False': False})

### Simple Columns

In [ ]:
sval = sval.drop(columns= ['Drop_Record_Index', 'Additional_Metics'] )

In [ ]:
# to bool
sval['phno'] = sval['phno'].astype('bool')
sval['soil'] = sval['soil'].astype('bool')
sval['meta'] = sval['meta'].astype('bool')

# to string
sval = cols_astype_string(
    df = sval, 
    col_list = [key for key in sval_col_dtypes.keys() if sval_col_dtypes[key] == 'string'])

sval.Year = year_string
sval.Year = sval.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = sval, dtype_dct = sval_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])


118/118 Columns pass.


## Weather

### Datetime

In [ ]:
# ... or we use the fields in the df to make a consistent format
wthr = cols_astype_string(
    df = wthr, 
    col_list = ['Year', 'Month', 'Day', 'Time'])

wthr['Datetime_Temp'] = wthr['Year']+'-'+wthr['Month']+'-'+wthr['Day']+' '+wthr['Time']

wthr['Datetime'] = pd.to_datetime(pd.Series(wthr.Datetime_Temp))
wthr = wthr.drop(columns= 'Datetime_Temp')

### Simple Columns 

In [ ]:
# to bool
wthr = sanitize_col(
    df = wthr, 
    col = 'Data_Cleaned', 
    simple_renames= {
        'Yes':'True',
         'No':'False'}, 
    split_renames= {})

# set missing to false
wthr.loc[wthr.Data_Cleaned.isna(), 'Data_Cleaned'] = 'False'
wthr.Data_Cleaned = wthr.Data_Cleaned.map({'True': True, 'False': False})

In [ ]:
# to string
wthr = cols_astype_string(
    df = wthr, 
    col_list = [key for key in wthr_col_dtypes.keys() if wthr_col_dtypes[key] == 'string'])

wthr.Year = year_string
wthr.Year = wthr.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = wthr, dtype_dct = wthr_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])

28/28 Columns pass.


## Management

### Date_Datetime

In [ ]:
mgmt['Date_Datetime'] = pd.to_datetime(mgmt['Date_Datetime'])

In [ ]:
# convert types
err_list = find_unconvertable_datetimes(df_col=mgmt.Date_Datetime, pattern='%m/%d/%y', index=False)
if err_list != []:
    print(err_list)
else:
    mgmt.Date_Datetime = pd.to_datetime(pd.Series(mgmt.Date_Datetime), format = '%m/%d/%y', errors='coerce')

### Amount_Per_Acre

In [ ]:
mask = (mgmt.Amount_Per_Acre == '50 GPA')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['50', 'GPA']

mask = (mgmt.Amount_Per_Acre == '4.5 pt./A')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['4.5', 'pt']

mask = (mgmt.Amount_Per_Acre == '60 gallons/acre')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['60', 'Gal']

mask = (mgmt.Amount_Per_Acre ==  '1/2 ton/A')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['0.5', 'Ton']

mask = (mgmt.Amount_Per_Acre ==  '1.00%')
mgmt.loc[mask, ['Amount_Per_Acre'] ] = [np.nan]

mask = (mgmt.Amount_Per_Acre ==  '1.5 qt./A')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['1.5', 'qt']

mask = (mgmt.Amount_Per_Acre ==  '.75 inch')
mgmt.loc[mask, ['Amount_Per_Acre', 'Unit'] ] = ['0.75', 'in']

mask = (mgmt.Amount_Per_Acre ==  '0.50%')
mgmt.loc[mask, ['Amount_Per_Acre'] ] = [np.nan]

In [ ]:
mgmt = sanitize_col(
    df = mgmt, 
    col = 'Amount_Per_Acre', 
    simple_renames= {
        '375 lbs./A': '375', 
        '30 Actual P': '30',  
        '91 lb': '91', 
        '100 Actual N': '100 (N)', 
        '2 lbs./A': '2', 
        '134-0-0': '134 (N)',  
        '180 N': '180 (N)', 
        '90 Actual K': '90 (K)', 
        '60 Actual N': '60 (N)', 
        '6.5 lbs./A': '6.5'
    }, 
    split_renames= {'150+5 Zn': ['16.5 (N)', '55.5 (P)', '5 (Zn)'], # based on product, calc to elements [(e/100)*150 for e in [11, 37, 0] ]
                    '20-9-3': ['20 (N)', '9 (P)', '3 (K)'], 
                    'NPK 27,26,0\n': ['27 (N)', '26 (P)'], 
                    '7.5 N, 30 P, 30 K':  ['7.5 (N)', '30 (P)', '30 (K)']
                   })

In [ ]:
mgmt = safe_create_col(mgmt, "Ingredient")
mask = mgmt.Ingredient.isna()
mgmt.loc[mask, 'Ingredient'] = mgmt.loc[mask, 'Product']

# assume each string is formated as 'val (key)'. `sanitize_col` should be used to enforce this.
for e in ['26 (P)', '60 (N)', '30 (K)', '7.5 (N)', '16.5 (N)', '3 (K)', '180 (N)', '134 (N)', '27 (N)', '30 (P)', '55.5 (P)', '100 (N)', '9 (P)', '5 (Zn)', '20 (N)', '90 (K)']:
    val = re.findall('^\d+[.]*\d*', e)[0]
    key = re.findall('\(.+\)',      e)[0].replace('(', '').replace(')', '')
    
    mask = (mgmt['Amount_Per_Acre'] == e)
    mgmt.loc[mask, 'Ingredient'] = key
    mgmt.loc[mask, 'Amount_Per_Acre'] = val

In [ ]:
# convert types
err_list = find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = False)
if err_list != []:
    print(err_list)
else:
    mgmt.Amount_Per_Acre = pd.to_numeric(mgmt.Amount_Per_Acre, errors='coerce')

### Ingredient
This is to be the cleaned up version of the "Product" column

In [ ]:
# list(mgmt.loc[:, 'Ingredient'].drop_duplicates())

### Simple Columns

In [ ]:
mgmt = mgmt.drop(columns= ['Drop_Record_Index', 'Drop_Record_Index2'])

In [ ]:
# to bool
mgmt['mgmt'] = mgmt['mgmt'].astype('bool')

# to string
for e in [ee for ee in ['Application', 'Product', 'Ingredient', 'Unit', 'Imputation_Notes',
                       'Replicate'] if ee in mgmt.columns]:
    mgmt[e] = mgmt[e].astype('string')

mgmt.Year = year_string
mgmt.Year = mgmt.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = mgmt, dtype_dct = mgmt_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])


13/13 Columns pass.


,Column,dtype,Expected_dtype,Pass


# Publish



In [ ]:
write_out_pkl(obj = sval, path = './data/interim/'+year_string+'sval.pickle')
write_out_pkl(obj = wthr, path = './data/interim/'+year_string+'wthr.pickle')
write_out_pkl(obj = mgmt, path = './data/interim/'+year_string+'mgmt.pickle')